In [1]:
import numpy as np
import pandas as pd

from ols_bootstrap.pairs import PairsBootstrap
from ols_bootstrap.residual import ResidualBootstrap
from ols_bootstrap.wild import WildBootstrap

In [2]:
df = pd.read_csv('./balance2018.csv')
df = df[df['sales_clean'] != 0]
df = df[['sales_clean', 'tanass_clean', 'tax']]
df = df.dropna(subset=['tanass_clean', 'tax'])

df_scaled = df.applymap(lambda x: np.log(x + 1))

/home/phamv/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (66) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df_sample = df_scaled.sample(n=10000, replace=False)

Y_data = pd.DataFrame(df_sample.iloc[:, 0])
X_data = pd.DataFrame(df_sample.iloc[:, 1:])

In [4]:
psb = PairsBootstrap(Y_data, X_data, reps = 1000, se_type="hc5", ci_type="bca")
psb.fit()

In [5]:
rsb = ResidualBootstrap(Y_data, X_data, reps = 1000, se_type="hc5", ci_type="bca")
rsb.fit()

In [6]:
wb_stdn = WildBootstrap(Y_data, X_data, reps = 1000, se_type="hc5", ci_type="bca", from_distro = "standard_normal")
wb_stdn.fit()

In [7]:
wb_rad = WildBootstrap(Y_data, X_data, reps = 1000, se_type="hc5", ci_type="bca", from_distro = "rademacher")
wb_rad.fit()

In [8]:
wb_mam = WildBootstrap(Y_data, X_data, reps = 1000, se_type="hc5", ci_type="bca", from_distro = "mammen")
wb_mam.fit()

In [9]:
psb.summary()

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                           Pairs Bootstrap results with sample size of 10000 and bootstrap resampling size of 1000 using HC5 SE-s with 95.00% BCa CI                            |
+----------------+---------------+-------------------------------+----------+-----------------+-----------------------------+-------------------+--------------------+-----------+
|     Params     |  Orig Coeffs  |  Mean of Bootstrapped Coeffs  |   Bias   |  Orig Coeff SE  |  SE of Bootstrapped Coeffs  |  % of Diff in SE  |         CI         |  CI Diff  |
+----------------+---------------+-------------------------------+----------+-----------------+-----------------------------+-------------------+--------------------+-----------+
|     const      |     6.2193    |             6.2159            |  0.0034  |      0.0456     |          

In [10]:
rsb.summary()

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                          Residual Bootstrap results with sample size of 10000 and bootstrap resampling size of 1000 using HC5 SE-s with 95.00% BCa CI                          |
+----------------+---------------+-------------------------------+----------+-----------------+-----------------------------+-------------------+--------------------+-----------+
|     Params     |  Orig Coeffs  |  Mean of Bootstrapped Coeffs  |   Bias   |  Orig Coeff SE  |  SE of Bootstrapped Coeffs  |  % of Diff in SE  |         CI         |  CI Diff  |
+----------------+---------------+-------------------------------+----------+-----------------+-----------------------------+-------------------+--------------------+-----------+
|     const      |     6.2193    |             6.2167            |  0.0026  |      0.0456     |          

In [11]:
wb_stdn.summary()

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                 Wild Bootstrap with Standard Normal results with sample size of 10000 and bootstrap resampling size of 1000 using HC5 SE-s with 95.00% BCa CI                  |
+----------------+---------------+-------------------------------+----------+-----------------+-----------------------------+-------------------+--------------------+-----------+
|     Params     |  Orig Coeffs  |  Mean of Bootstrapped Coeffs  |   Bias   |  Orig Coeff SE  |  SE of Bootstrapped Coeffs  |  % of Diff in SE  |         CI         |  CI Diff  |
+----------------+---------------+-------------------------------+----------+-----------------+-----------------------------+-------------------+--------------------+-----------+
|     const      |     6.2193    |             6.2208            |  0.0015  |      0.0456     |          

In [12]:
wb_rad.summary()

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                    Wild Bootstrap with Rademacher results with sample size of 10000 and bootstrap resampling size of 1000 using HC5 SE-s with 95.00% BCa CI                    |
+----------------+---------------+-------------------------------+----------+-----------------+-----------------------------+-------------------+--------------------+-----------+
|     Params     |  Orig Coeffs  |  Mean of Bootstrapped Coeffs  |   Bias   |  Orig Coeff SE  |  SE of Bootstrapped Coeffs  |  % of Diff in SE  |         CI         |  CI Diff  |
+----------------+---------------+-------------------------------+----------+-----------------+-----------------------------+-------------------+--------------------+-----------+
|     const      |     6.2193    |             6.2184            |  0.0009  |      0.0456     |          

In [13]:
wb_mam.summary()

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                      Wild Bootstrap with Mammen results with sample size of 10000 and bootstrap resampling size of 1000 using HC5 SE-s with 95.00% BCa CI                      |
+----------------+---------------+-------------------------------+----------+-----------------+-----------------------------+-------------------+--------------------+-----------+
|     Params     |  Orig Coeffs  |  Mean of Bootstrapped Coeffs  |   Bias   |  Orig Coeff SE  |  SE of Bootstrapped Coeffs  |  % of Diff in SE  |         CI         |  CI Diff  |
+----------------+---------------+-------------------------------+----------+-----------------+-----------------------------+-------------------+--------------------+-----------+
|     const      |     6.2193    |             6.2201            |  0.0008  |      0.0456     |          